# 01 - Single Agent from Personal Data

Can we create an LLM persona from personal data that responds to surveys like the real person?

This notebook tests the idea with a simple approach:
1. Load personal context (CV, writing samples, etc.)
2. Create a persona from that context
3. Ask it questions
4. Check if the answers match reality

In [1]:
# Setup - ensure we can import from src
import sys
sys.path.insert(0, '../src')

from centuria.models import Persona, Question, Survey
from centuria.persona import create_persona
from centuria.survey import ask_question, run_survey, estimate_survey_cost

## Step 1: Load Context

The persona needs information about the person. More relevant context = better responses. Lets try with my CV:

In [2]:
from centuria.data import load_files

my_context = load_files(['../data/personal/cv.pdf'])

In [3]:
# Create the persona
persona = create_persona(
    name="My Persona",
    context=my_context
)

print(f"Created persona: {persona.name}")
print(f"Context length: {len(persona.context.split())} words")

Created persona: My Persona
Context length: 1149 words


## Understanding the Prompts

Before asking questions, let's see what prompts are being sent to the LLM. This is where prompt engineering improvements can be made to create more accurate personas.

In [4]:
from centuria.survey import (
    SYSTEM_TEMPLATE,
    USER_TEMPLATE_SINGLE_SELECT,
    USER_TEMPLATE_OPEN_ENDED,
    build_system_prompt,
    build_user_prompt,
)

print("=" * 60)
print("SYSTEM PROMPT TEMPLATE")
print("=" * 60)
print(SYSTEM_TEMPLATE)
print("\n")
print("=" * 60)
print("USER PROMPT TEMPLATE (Single Select)")
print("=" * 60)
print(USER_TEMPLATE_SINGLE_SELECT)
print("\n")
print("=" * 60)
print("USER PROMPT TEMPLATE (Open Ended)")
print("=" * 60)
print(USER_TEMPLATE_OPEN_ENDED)

SYSTEM PROMPT TEMPLATE
You are role-playing as {name}. Answer all questions as this person would, based on the context provided.

<context>
{context}
</context>

Guidelines:
- Respond authentically as this person based on the context
- Draw on the context to inform your answers, preferences, and opinions
- If the context doesn't cover something, make reasonable inferences consistent with what you know about this person
- Stay in character throughout


USER PROMPT TEMPLATE (Single Select)
Question: {question}

Options: {options}

Reply with ONLY the option you choose, nothing else.


USER PROMPT TEMPLATE (Open Ended)
Question: {question}

Provide a brief response.


### Example: What the LLM actually sees

Here's what the filled-in prompts look like for this persona. The system prompt contains the persona context, while the user prompt contains just the question.

**Areas for optimization:**
- System prompt: Role-playing instructions, context formatting, guidelines for handling gaps
- User prompt: Question framing, response format instructions
- Context: What personal data to include, how to structure it

In [5]:
# Example question to demonstrate the prompts
example_q = Question(
    id="example",
    text="Which programming language do you prefer?",
    question_type="single_select",
    options=["Python", "JavaScript", "Rust", "Go"]
)

system_prompt = build_system_prompt(persona)
user_prompt = build_user_prompt(example_q)

print("=" * 60)
print("SYSTEM PROMPT (sent once per conversation)")
print("=" * 60)
print(system_prompt[:500] + "..." if len(system_prompt) > 500 else system_prompt)
print(f"\n[... {len(system_prompt)} total characters ...]")
print("\n")
print("=" * 60)
print("USER PROMPT (sent for each question)")
print("=" * 60)
print(user_prompt)

SYSTEM PROMPT (sent once per conversation)
You are role-playing as My Persona. Answer all questions as this person would, based on the context provided.

<context>
SEAN GREAVES Email: seanwgreaves@gmail.com 
GitHub: github.com/ribenamaplesyrup 
Portfolio: seangreaves.xyz 
EMPLOYMENT  
 
APPLIED AI ENGINEER - THE AUTONOMY INSTITUTE (MAY 2023 - PRESENT) 
A leading think-tank developing data-driven tools for sustainable economic planning  
• Led the institute's strategic exploration of generative AI, establishing a new applied AI research c...

[... 8625 total characters ...]


USER PROMPT (sent for each question)
Question: Which programming language do you prefer?

Options: Python, JavaScript, Rust, Go

Reply with ONLY the option you choose, nothing else.


## Step 2: Ask Single Questions

Test the persona with individual questions before running a full survey.

In [6]:
# Single select question
q1 = Question(
    id="q1",
    text="Which programming language do you prefer?",
    question_type="single_select",
    options=["Python", "JavaScript", "Rust", "Go"]
)

response = await ask_question(persona, q1)
print(f"Question: {q1.text}")
print(f"Response: {response.response}")

Question: Which programming language do you prefer?
Response: Python


Now what about an open ended question?

In [7]:
# Open-ended question
q2 = Question(
    id="q2",
    text="What motivates you in your work? Answer in less than 20 words.",
    question_type="open_ended"
)

response = await ask_question(persona, q2)
print(f"Question: {q2.text}")
print(f"Response: {response.response}")

Question: What motivates you in your work? Answer in less than 20 words.
Response: I'm motivated by advancing AI for societal good, fostering transparency, and driving impactful policy and technological innovation.


A crucial piece of data we've skipped over here is **cost**. Its one of the key components within the business case for why we might build around using AI persona's over human personas. If cost was super high, we might opt to use humans, but of course cost is very low!

In [8]:
print(f"\n--- Cost ---")
print(f"Tokens: {response.prompt_tokens:,} prompt + {response.completion_tokens:,} completion")
print(f"Cost: ${response.cost:.6f}")


--- Cost ---
Tokens: 1,808 prompt + 20 completion
Cost: $0.002480


However its perhaps more powerful to estimate how much a query would cost before we send it. That way we can clearly scope the cost of running a survey on a specific number of agents...

In [10]:
# Estimate cost for a single question BEFORE sending it
from centuria.llm import estimate_cost

system = build_system_prompt(persona)
user = build_user_prompt(q2)

estimate = estimate_cost(user, system=system, estimated_completion_tokens=20)

print("COST ESTIMATE (before sending)")
print("="*60)
print(f"Prompt tokens:               {estimate.prompt_tokens:,}")
print(f"Est. completion tokens:      {estimate.completion_tokens:,}")
print(f"Est. cost:                   ${estimate.cost:.6f}")

# Now actually run it and compare
actual_response = await ask_question(persona, q2)

print(f"\nACTUAL COST (after sending)")
print("="*60)
print(f"Prompt tokens:               {actual_response.prompt_tokens:,}")
print(f"Completion tokens:           {actual_response.completion_tokens:,}")
print(f"Actual cost:                 ${actual_response.cost:.6f}")

COST ESTIMATE (before sending)
Prompt tokens:               1,808
Est. completion tokens:      20
Est. cost:                   $0.004760

ACTUAL COST (after sending)
Prompt tokens:               1,808
Completion tokens:           18
Actual cost:                 $0.002460


**Why is the actual cost lower than the estimate?**

The estimate assumes full price for all prompt tokens, but API providers like Anthropic and OpenAI use **prompt caching** server-side. Since the same system prompt (your CV context) was already sent earlier in this notebook, the provider caches that prefix and charges a reduced rate for subsequent requests.

This is good news for the use case: running the same persona against many questions means the context gets cached, and each additional question costs less than the first.

**Note:** This is different from LiteLLM's local caching, which can return identical results at zero cost if you re-run the exact same prompt. The provider-side caching still makes API calls but at reduced input token rates.

## Step 3: Run a Survey

Run multiple questions together as a survey.

In [11]:
# Define the survey
mini_survey = Survey(
    id="persona_validation",
    name="Persona Validation Survey",
    questions=[
        # === CV-INFERABLE QUESTIONS ===
        Question(
            id="career_priority",
            text="What matters most to you in a job?",
            question_type="single_select",
            options=["Impact on society", "Financial compensation", "Learning opportunities", "Work-life balance", "Autonomy and creativity"]
        ),
        Question(
            id="tech_stance",
            text="How do you feel about AI's impact on employment?",
            question_type="single_select",
            options=["Net positive - creates more jobs than it destroys", "Net negative - mass displacement is coming", "Neutral - it will transform jobs but balance out", "Too early to tell"]
        ),
        Question(
            id="work_style",
            text="How do you prefer to approach complex problems?",
            question_type="single_select",
            options=["Deep solo research then collaborate", "Immediate team brainstorming", "Build a prototype first, discuss later", "Map out the theory before any implementation"]
        ),
        Question(
            id="industry_interest",
            text="Which sector would you most like to work in?",
            question_type="single_select",
            options=["Climate/sustainability tech", "Healthcare/biotech", "Finance/fintech", "Government/public sector", "Pure research/academia"]
        ),
        Question(
            id="skill_development",
            text="If you had 3 months to learn anything, what would you prioritise?",
            question_type="single_select",
            options=["Technical depth (new frameworks, languages)", "Domain expertise (economics, biology, etc.)", "Leadership and management skills", "Creative skills (writing, design)", "Starting a business"]
        ),
        
        # === CULTURAL/POLITICAL QUESTIONS ===
        Question(
            id="ubi_stance",
            text="What's your view on Universal Basic Income?",
            question_type="single_select",
            options=["Strongly support - essential for the future", "Cautiously support - worth piloting", "Skeptical - prefer targeted interventions", "Oppose - undermines work incentives"]
        ),
        Question(
            id="privacy_tradeoff",
            text="How do you feel about trading personal data for free services?",
            question_type="single_select",
            options=["Acceptable - fair exchange", "Uncomfortable but unavoidable", "Strongly oppose - privacy is sacred", "Depends entirely on what data and what service"]
        ),
        Question(
            id="institutions_trust",
            text="Which institution do you trust most to act in the public interest?",
            question_type="single_select",
            options=["National government", "Local government", "Large corporations", "Non-profits and NGOs", "Academic institutions", "None of the above"]
        ),
        Question(
            id="optimism_future",
            text="How do you feel about the next 20 years for humanity?",
            question_type="single_select",
            options=["Very optimistic - best time to be alive", "Cautiously optimistic - progress will continue", "Anxious - major challenges ahead", "Pessimistic - decline seems likely"]
        ),
        Question(
            id="controversy_take",
            text="Which controversial opinion are you most sympathetic to?",
            question_type="single_select",
            options=["Most meetings should be emails", "Remote work is strictly better than office work", "Cryptocurrency has no legitimate use case", "Social media does more harm than good", "Economic growth should not be the primary goal"]
        ),
    ]
)

# Estimate cost for the full survey
survey_estimate = estimate_survey_cost(persona, mini_survey, num_agents=1)

print("SURVEY COST ESTIMATE")
print("="*60)
print(f"Questions:                   {len(mini_survey.questions)}")
print(f"Prompt tokens per agent:     {survey_estimate.prompt_tokens:,}")
print(f"Est. completion tokens:      {survey_estimate.completion_tokens:,}")
print(f"Est. cost per agent:         ${survey_estimate.cost_per_agent:.4f}")
print()
print("Scale projections:")
for n in [10, 100, 1000]:
    scaled = estimate_survey_cost(persona, mini_survey, num_agents=n)
    print(f"  {n:>5} agents: ${scaled.total_cost:.2f}")

SURVEY COST ESTIMATE
Questions:                   10
Prompt tokens per agent:     18,417
Est. completion tokens:      50
Est. cost per agent:         $0.0469

Scale projections:
     10 agents: $0.47
    100 agents: $4.69
   1000 agents: $46.92


In [12]:
# Run the survey (this is where the API calls happen)
survey_response = await run_survey(persona, mini_survey)

print(f"Survey: {mini_survey.name}")
print(f"\n{'='*60}")
print("CV-INFERABLE QUESTIONS")
print('='*60)
for r in survey_response.responses[:5]:
    q = next(q for q in mini_survey.questions if q.id == r.question_id)
    print(f"\nQ: {q.text}")
    print(f"A: {r.response}")

print(f"\n{'='*60}")
print("CULTURAL/POLITICAL QUESTIONS (harder to infer)")
print('='*60)
for r in survey_response.responses[5:]:
    q = next(q for q in mini_survey.questions if q.id == r.question_id)
    print(f"\nQ: {q.text}")
    print(f"A: {r.response}")

print(f"\n{'='*60}")
print("ACTUAL COST")
print('='*60)
print(f"Total tokens: {survey_response.total_tokens:,}")
print(f"Total cost:   ${survey_response.total_cost:.4f}")

Survey: Persona Validation Survey

CV-INFERABLE QUESTIONS

Q: What matters most to you in a job?
A: Impact on society

Q: How do you feel about AI's impact on employment?
A: Neutral - it will transform jobs but balance out

Q: How do you prefer to approach complex problems?
A: Deep solo research then collaborate

Q: Which sector would you most like to work in?
A: Climate/sustainability tech

Q: If you had 3 months to learn anything, what would you prioritise?
A: Domain expertise (economics, biology, etc.)

CULTURAL/POLITICAL QUESTIONS (harder to infer)

Q: What's your view on Universal Basic Income?
A: Cautiously support - worth piloting

Q: How do you feel about trading personal data for free services?
A: Depends entirely on what data and what service

Q: Which institution do you trust most to act in the public interest?
A: Non-profits and NGOs

Q: How do you feel about the next 20 years for humanity?
A: Cautiously optimistic - progress will continue

Q: Which controversial opinion ar

## Step 4: Check Accuracy

Compare the persona's answers to your actual answers.

In [ ]:
# Your actual answers (fill these in to check accuracy)
my_answers = {
    # CV-inferable questions
    "career_priority": "",      # e.g., "Impact on society"
    "tech_stance": "",          # e.g., "Neutral - it will transform jobs but balance out"
    "work_style": "",           # e.g., "Build a prototype first, discuss later"
    "industry_interest": "",    # e.g., "Climate/sustainability tech"
    "skill_development": "",    # e.g., "Domain expertise (economics, biology, etc.)"
    
    # Cultural/political questions
    "ubi_stance": "",           # e.g., "Cautiously support - worth piloting"
    "privacy_tradeoff": "",     # e.g., "Depends entirely on what data and what service"
    "institutions_trust": "",   # e.g., "Academic institutions"
    "optimism_future": "",      # e.g., "Cautiously optimistic - progress will continue"
    "controversy_take": "",     # e.g., "Economic growth should not be the primary goal"
}

# Compare answers
cv_correct = 0
cv_total = 0
cultural_correct = 0
cultural_total = 0

cv_questions = ["career_priority", "tech_stance", "work_style", "industry_interest", "skill_development"]

print("COMPARISON: Persona vs Reality")
print("="*60)

for r in survey_response.responses:
    if r.question_id in my_answers and my_answers[r.question_id]:
        q = next(q for q in mini_survey.questions if q.id == r.question_id)
        match = r.response == my_answers[r.question_id]
        
        if r.question_id in cv_questions:
            cv_total += 1
            if match:
                cv_correct += 1
        else:
            cultural_total += 1
            if match:
                cultural_correct += 1
        
        status = "✓" if match else "✗"
        print(f"\n{status} {q.text}")
        print(f"   Persona: {r.response}")
        print(f"   Actual:  {my_answers[r.question_id]}")

print(f"\n{'='*60}")
print("ACCURACY SUMMARY")
print("="*60)
if cv_total > 0:
    print(f"CV-inferable questions:  {cv_correct}/{cv_total} ({cv_correct/cv_total:.0%})")
if cultural_total > 0:
    print(f"Cultural/political:      {cultural_correct}/{cultural_total} ({cultural_correct/cultural_total:.0%})")
if cv_total + cultural_total > 0:
    total = cv_correct + cultural_correct
    all_total = cv_total + cultural_total
    print(f"Overall:                 {total}/{all_total} ({total/all_total:.0%})")

## Results

How well did the persona match reality?

See `01_notes.md` for detailed commentary on limitations and improvements.